## Simulation and post-processing of the turbulent flow over a periodic hill using `Nek5000`
> Saleh Rezaeiravesh and Philipp Schlatter <br />
  salehr@mech.kth.se, pschlatt@mech.kth.se <br />
  SimEx/FLOW, KTH Engineering Mechanics, Royal Institute of Technology, Stockholm, Sweden

This is a short tutorial for simulation of a 3D turbulent flow over a periodic hill and post-processing of the resulting statsitics. 
The streamwise, wall-normal, and spanwise coordinates are specified by $x$, $y$, and $z$, respectively. The flow is periodic in $z$ direction, and periodic boundary condition is applied at the inlet and outlet surfaces with normal vectors in $-x$ and $+x$ directions, respectively. 

## 1. Nek5000 source code

We use the KTH-Framework extension of `Nek5000` which can be cloned from [here](https://github.com/KTH-Nek5000/KTH_Framework). 

The source files of the periodic hill case can bedownloaded from [this example](https://github.com/KTH-Nek5000/KTH_Examples/tree/eac7d10cc1b03278874f2a98a9a691b165dee4f0/phill_STAT).

## 2. Flow simulation

The case is called `phill`. In the following, different steps are briefly explained to set up and run the simulation as well as computing turbulence statsitics. 

### 2.1. Mesh Generation & Specifying Boundary Conditions

We use the internal mesh generator of `Nek5000` to create a uniform mesh for a 3D square box. Then, in the `.usr` file, the coordinates of the points of the geometry and mesh are adjusted at the first time step to obtain the geomtery of the periodic hill with a structured mesh. 

- In `phill.box`, the size of the box, and mesh size are set:
  * Line 17: Number of elements in $x$, $y$, and $z$ directions. 
  * Lines 18-20: Starting location, final location, and grading of the mesh elements in $x$, $y$, and $z$. 
  * Line 21: Boundary conditions of the six faces of the box with normals in $-x$, $+x$, $-y$, $+y$, $-z$, and $+z$ are respectively set as: `P  ,P  ,W  ,W  ,P  ,P  `, where, `W`=wall, `P`=periodic. 
  
- To create the box and its mesh: run `genbox` which is a `Nek5000` standard tool. This results in `box.re2`. Rename this file as `phill.re2`.

- Map the mesh by `genmap` and create `phill.ma2`. 

When `Nek5000` is compiled and run, the box is mapped into a 3D periodic hill. This is achieved by calling subroutine `box2phill` in `usrdat2` in `phill.usr`. The dimension of the periodic hill geomtery as well as the mesh stretching parameters in the wall-normal directions are specified in subtourine `usrdat2`. 

Since the flow is periodic in the streamwise direction, a forcing term is required to keep the mass flow rate constant at each time step during the simulation. The associated bulk velocity (streamwise velocity averaged over the volume of the whole domain) is computed in subroutine `usrdat3`. 

<img src="./static_figs/phill_mesh.png" alt="drawing" width="600"/>



<center>The mesh created by Nek5000 for the periodic hill geometry.</center>


### 2.2. Compilation

- First, the following should be set in `SIZE`:
    * `lx1`: Number of Gauss-Lobatto-Legendre (GLL) points per element per dimension. 
    * `lxd`: Number of quadrature points for dealiasing (over-integration). We take the integer part of $lxd=3/2\, lx1$.
    * `lelg`: Total number of elements which is obtained from what is set in `phill.box`. This parameter is allowed to be set to a larger value than the actual number of elements, but not less! 
    * `lelt`: Maximum total number of elements per MPI rank. This value can be set considering the minimum MPI rank to be used to run the case. 
    The rest of parameters are kept as default. 

- Next, we need to set the required parameters in the compilation script `compile_script`:
    * Set the absolute path to `KTH_Toolbox` in `TOOLBOX_SRC`. 
    * Set the absolute path to `Nek5000` source code in `NEK_SOURCE_ROOT`.
    * Set appropriate FORTRAN and C compilers to `FC` and `CC`, respectively. The default is `FC=mpifort` and `CC=mpicc`. 
    
To compile, run `./compile_script --all`. The compilation report will be written in `build.log`.

For cleaning old compilation files, run `./compile_script --clean` and then type `Y`.     

### 2.3. Running the case

After compiling the case, the object file `nek5000` is created which can be executed. To run the case with, for instance, 8 processors, type

`mpirun -np 8 nek5000 >>logfile&`

However, prior to this, two things should be done:

1. Create a file named `SESSION.NAME`. The first line is the case name, i.e. `phill` and the second line is the absolute path to the case directory (do not forget `/` at the end of the path).

2. Set the simulation parameters in `phill.par`. The parameters include:
   * Maximum number of time steps `numSteps`
   * Time step size, `dt`. For a constant $\Delta t$ set a negative value
   * Interval of saving flow fields `writeInterval`
   * Filtering parameters
   * Checkpointing parameters under `[_CHKPOINT]` to back up and restart simulation
   * Sampling frequency for time averaging`AVSTEP` and intervals of writing averaged fields, `IOSTEP`
   * Reynolds number or viscosity, `viscosity`. If a negative value is set, it will be considered as the Reynolds number. 

List of the output files:
   * `phill0.f*`: snapshots of the flow fields written at intervals `writeInterval`.
   * `rs6phill0.f*`: backup files required for restarting the simulation. These files are written with frequency `CHKPINTERVAL` set in `phill.par`.     
   * `stsphill0.f*`: averaged fields to be used for extracting turbulence statistics. 

**Contour plots of the snapshots:**
The snapshot fields `phill0.f*` can be plotted using [`ParaView`](https://www.paraview.org/) or [`VisIt`](https://wci.llnl.gov/simulation/computer-codes/visit). To this end, we need to first run `visnek` command in the case root. This creates `phill.nek5000` which can be opened in the mentioned software. 

<img src="./static_figs/phill_vel.png" alt="drawing" width="500"/>

<center>Contours of an instantaneous field of velocity magnitude.</center>

## 3. Post-processing the Turbulence Statistics

The flow statistics are computed by averaging over time and the $z$-direction. As a result, `stsphill0.f*` files are created which contain various fields required to compute turbulence statistics.The final statistics are 2D and vary with $x$ and $y$ directions.

To obtain the post-processed 2D turbulence statistics, the following steps are required:

1. **Step 1** - Flow simulation and gathering the statistics: <br />
This needs running the `Nek5000` case [`phill_STAT`](https://github.com/KTH-Nek5000/KTH_Examples/tree/master/phill_STAT). By gathering the statistics (averaged over time and z-direction), two sets of files are produced: `stsphill0.f*` and `c2Dphill0.f*`. 

&nbsp;

2. **Step 2** - Merge the statistics data files and interpolate the data on the interpolation points:<br />
These are achieved by running `Nek5000` in [`phill_PSTAT2D/pp_Nek/`](https://github.com/KTH-Nek5000/KTH_Examples/tree/master/phill_PSTAT2D/pp_Nek), see below for the details. As a result of this step, `int_fld` is produced in `/pp_Nek/DATA/`. Note that the interpolation is from the GLL points in the simulation grid to a set of points specified in `int_pos`. For the list of fields written in `int_fld`, see [this list](https://github.com/KTH-Nek5000/KTH_Examples/blob/master/pipe_PSTAT2D/pp_python/nom_fields.txt).

&nbsp;

3. **Step 3** - Extract mean, RMS, and other turbulence statistics from `int_fld`:<br />
As detailed below, by running a set of `Python` scripts located in [`phill_PSTAT2D/pp_Python/`](https://github.com/KTH-Nek5000/KTH_Examples/tree/master/phill_PSTAT2D/pp_Python), the profiles of turbulence statistics on the interpolation points are constructed from the fields available in `int_fld`.

### 3.1. Details of Step 2
* Change directory to `/phill_PSTAT2D/`.
* In `./pp_Python/modules/` run `writer_int_pos.py` to create `int_pos` which contains a set of points at which the gathered statistics are to be interpolated. `int_pos` will be created in `./pp_Nek/DATA/`. Before running the script, the parameters in `./pp_Python/inputs_phill_pp.in` should be set. The "Fluid Properties" and "Hill Geometry" parameters should be the same as what used in the flow simulation (i.e. in Step 1). Currently two sets of interpolation points are considered: 
  * (P-i) A set of streamwise locations `xpvrt` is considered at each `npvrt` points are created in the y-direction between the two walls. 
  * (P-ii) Adjacent to the lower wall and between the two hills,  `npwall` points are created in the streamwise direction.

* Copy `stsphill0.f*` and `c2dphill0.f*` files to `./pp_Nek/DATA/`.

* Run `Nek5000` to create `int_fld` in `../pp_Nek/DATA/`. In `../pp_Nek/` do these:
    - Set `lx1` and `lxd` in `SIZE` based on what is used in the flow simulation.
    - Set `lhis` in `SIZE` such that it is at least the same of as the total number of the interpolation points.
    - Set `viscosity` and `density` in `phill.par` as what is used in the simulation. Also, set the post-processing parameters under `[_PSTAT2D]`.
    - Compile the case by `./compile_script --all` and then run `nek5000`.
As a result, `int_fld` is generated which should then be moved to `./DATA/`.

### 3.2. Details of Step 3
To extract the profiles of the turbulence statistics, the `Python` scripts in `./pp_Python/` are used:

  * The whole workflow is implemented in `main_pHill_PP.py`. Therefore, one can only run `python main_pHill_PP`. For different functions called in `main_pHill_PP.py`, see the scripts in `/modules/`.
  * The input parameters and information about the interpolation points are read from `/pp_Python/inputs_phill_pp.in`. 
  * The resulting database `dbProfs` and `dbWall` contain the turbulence statistics at the (P-i) and (P-ii) interpolation points, respectively, see above. For the complete set of quantities in these databases, print `dbProfs.keys()`.
  * A set of example plots are provided at the end of `main_pHill_PP.py`. Note that in addition to the statistical moments of flow variables, one can plot various budget terms for the Reynolds stress components. 

<center>Profiles of $\langle u \rangle$ at different streamwise locations.</center>

<img src="./static_figs/phill_uProfs.png" alt="drawing" width="800"/>

<center>Profiles of turbulent kinetic energy (TKE) at different streamwise locations.</center>
<img src="./static_figs/phill_tkeProfs.png" alt="drawing" width="800"/>

<center>Profiles of $\langle u'v' \rangle$ at different streamwise locations.</center>
<img src="./static_figs/phill_uvProfs.png" alt="drawing" width="800"/>